In [1]:
import pandas as pd
import numpy as np
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os
import plotly.figure_factory as ff
import plotly.express as px
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'congress'
))

In [3]:
myquery = '''
SELECT directordername, bioguideid
FROM members
ORDER BY lastname
'''

members = pd.read_sql_query(myquery, con=engine)
memberlist = [{'label': x, 'value': y} 
 for x, y in zip(members['directordername'], members['bioguideid'])]

In [4]:
myquery = '''
SELECT bioguideid,
    directordername, 
    party,
    ideology,
    state,
    district
FROM members m
'''

ideo_df = pd.read_sql_query(myquery, con=engine)
ideo_df['chamber'] = ideo_df['district'].isnull()
ideo_df['chamber'] = ideo_df['chamber'].replace({True: 'Senate', False: 'House of Representatives'})

In [5]:
ideo_df['namedistrict'] = [n + ' (' + s + ')' if c == 'Senate' 
    else n + ' (' + s + '-' + str(int(d)) + ')' 
    for n, s, c, d in zip(ideo_df['directordername'], ideo_df['state'], ideo_df['chamber'], ideo_df['district'])
]
ideo_df['party'] = ideo_df['party'].replace({'D': 'Democrat',
                                            'I': 'Independent',
                                            'R': 'Republican'})

userbioguideid = 'G000595'
ideo2 = ideo_df.query(f"bioguideid == '{userbioguideid}'")

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [7]:
mymarkdown = '''
Congressional elections are usually all about the big national issues. However, these Representatives and Senators work for local districts and states. Not all issues are the same everywhere. The purpose of this dashboard is to collect public data from these sources: 

* [Official API for the U.S. Congress](https://api.congress.gov/)
* [Voteview](https://voteview.com/)
* [Open Secrets](https://www.opensecrets.org/open-data/api)
* [ProPublica] (https://www.propublica.org/datastore/dataset/congressional-data-bulk-legislation-bills)

'''

In [13]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        # Stuff on top
        html.H1("Know Your Representatives in Congress"),
        html.H2("Data collected from APIs from Congress.gov, Open Secrets, Voteview.com, and Propublica"),
        html.H3("DS 6600: Data Engineering 1, UVA Data Science"),
        
        # Side bar
        html.Div([
            dcc.Markdown('Please select a Representative or Senator'),
            dcc.Dropdown(id = 'member', options = memberlist, value = 'A000370'),
            dcc.Markdown(mymarkdown),
        ], style = {'width': '24%', 'float': 'left'}),
        
        # Main bar
        html.Div([
            dcc.Tabs([
                 dcc.Tab(label = 'Biographical Info', children = [
                    # stuff for bio tab goes here
                    html.Div([html.Img(id = 'bioimage', style={'height':'100%', 'width':'100%'})], style = {'width': '24%', 'float':'left'}),
                    html.Div([dcc.Graph(id = 'biotable')], style = {'width': '74%', 'float':'right'})
                ]),
                
                dcc.Tab(label = 'Bill Sponsorship', children = [
                    # stuff for bill sponsorsihp tab goes here
                ]),
                
                dcc.Tab(label = 'Voting and Ideology', children = [
                    dcc.Graph(id = 'ideograph', style = {'height': '100%', 'width': '100%'})
                ]),
                
                dcc.Tab(label = 'Donors and Financial Info', children = [
                    # stuff for donors and financial info tab goes here
                ]),
                
            ])

            
        ], style = {'width': '74%', 'float': 'right'})



    ]
)

@app.callback([Output(component_id = 'biotable', component_property = 'figure')],
             [Input(component_id = 'member', component_property = 'value')]) 


def biotable(b):
    myquery = f'''
    SELECT directordername AS Name,
        party AS Party,
        state AS State,
        CAST(district as int) AS District,
        addressinformation_officeaddress AS Address,
        CONCAT (addressinformation_city, ', ', addressinformation_district) AS City,
        addressinformation_zipcode AS Zipcode,
        addressinformation_phonenumber AS Phone
        
    FROM members 
    WHERE bioguideid='{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine) 
    mydf.columns = [x.capitalize() for x in mydf.columns]
    mydf = mydf.T.reset_index()
    mydf = mydf.rename({'index': '', 0:''}, axis = 1)
    ff.create_table(mydf)
    return [ff.create_table(mydf)]

@app.callback([Output(component_id = 'bioimage', component_property = 'src')],
             [Input(component_id = 'member', component_property = 'value')])

def bioimage(b):
    myquery = f'''
    SELECT depiction_imageurl
    FROM members
    WHERE bioguideid='{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine)
    return [mydf['depiction_imageurl'][0]]

@app.callback([Output(component_id = 'ideograph', component_property = 'figure')],
             [Input(component_id = 'member', component_property = 'value')])

def ideograph(b): 
    ideo2 = ideo_df.query(f"bioguideid == '{b}'")
    fig = px.scatter(ideo_df, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict', 
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress',
                    opacity = .1)
    
    fig.update(layout=dict(title=dict(x=0.5)))
    fig.update_yaxes(range=(-0.5, 1.5), tickvals=[0, 1],
                    ticktext = ['Senate', 'House of Representatives'])
    fig.update_xaxes(range=(-1,1), tickvals=[-1, -.5, 0, .5, 1],
                    ticktext = ['Extreme left', 'Moderate left', 'Centrist', 'Moderate right', 'Extreme right'])
    
     
    
    fig.add_traces(
        px.scatter(ideo2, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict',
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress').update_traces(marker=dict(size=8, symbol="star", line=dict(width=2, color="DarkSlateGrey")),
                                                                                showlegend=False).data
    )
    
     
    
    
    return [fig]

if __name__=="__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

In [9]:

def biotable(b):
    myquery = f'''
    SELECT directordername AS Name,
        party AS Party,
        state AS State,
        CAST(district as int) AS District,
        addressinformation_officeaddress AS Address,
        CONCAT (addressinformation_city, ', ', addressinformation_district) AS City,
        addressinformation_zipcode AS Zipcode,
        addressinformation_phonenumber AS Phone
        
    FROM members 
    WHERE bioguideid='{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine) 
    mydf.columns = [x.capitalize() for x in mydf.columns]
    mydf = mydf.T.reset_index()
    mydf = mydf.rename({'index': '', 0:''}, axis = 1)
    ff.create_table(mydf)
    return ff.create_table(mydf)